In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.geometry.gasp_based.fuselage import (
    BWBFuselageParameters1,
    BWBCabinLayout,
    BWBFuselageParameters2,
    BWBFuselageSize,
    BWBFuselageGroup,
    BWBFuselageSize,
)
from aviary.subsystems.geometry.gasp_based.engine import (
    EngineSize,
    BWBEngineSizeGroup,
    PercentNotInFuselage,
)

glue_variable(get_variable_name(BWBFuselageParameters1), md_code=True)
glue_variable(get_variable_name(BWBCabinLayout), md_code=True)
glue_variable(get_variable_name(BWBFuselageParameters2), md_code=True)
glue_variable(get_variable_name(BWBFuselageSize), md_code=True)
glue_variable(get_variable_name(BWBFuselageGroup), md_code=True)
glue_variable(get_variable_name(EngineSize), md_code=True)
glue_variable(get_variable_name(BWBEngineSizeGroup), md_code=True)
glue_variable(get_variable_name(PercentNotInFuselage), md_code=True)

glue_variable(get_variable_name(Aircraft.Wing.VERTICAL_MOUNT_LOCATION), md_code=True)

# GASP based BWB model implementation

Modeling of GASP based blended wing body (BWB) aircraft is similar to that of the conventional aircraft but with several key differences. This page is a record of the development. For users, please read [Blended Wing Body Modeling](../user_guide/features/blended_wing_body) in the user guide.

## BWB geometry subsystem

For BWB geomtric subsystem, we add the following components:
- {glue:md}`BWBFuselageParameters1`: This component computes several fuselage geometric parameters based on user inputs
- {glue:md}`BWBCabinLayout`: This component computes the layout of passenger cabin for BWB. Several parameters are hard coded in this component.
- {glue:md}`BWBFuselageParameters2`: This component computes several fuselage geometric parameters based on both the user inputs and cabin layout.
- {glue:md}`BWBFuselageSize`: It carries out the computation of fuselage length and wetted area of BWB model.
- {glue:md}`BWBFuselageGroup`: {glue:md}`BWBFuselageParameters1` + {glue:md}`BWBCabinLayout` + {glue:md}`BWBFuselageParameters2` + {glue:md}`BWBFuselageSize`
- {glue:md}`PercentNotInFuselage`: For BWB, engines may be partially buried into fuselage. This component computes the percentage of corresponding surface area of nacelles not buried in fuselage. This parameter is passed to {glue:md}`EngineSize` component which computes the wetted area of nacelle. This function has infinity derivatives at the two ends. We use two cubic polynomials to smooth it.
- {glue:md}`BWBEngineSizeGroup`: {glue:md}`PercentNotInFuselage` + {glue:md}`EngineSize`. For Tube+Wing aircraft, we assume that engines are not buried into fuselage and hence {glue:md}`EngineSize` is good enough. But this feature can be extended to conventional aircraft.
- {glue:md}`ExposedWing`: Computation of exposed wing area. This is useful for BWB, but is available to tube+wing model too. For {glue:md}`Aircraft.Wing.VERTICAL_MOUNT_LOCATION` in the range (0, 1), the function has infinity derivatives at the two ends. We use two cubic polynomials to smooth it.

In [ ]:
# Testing Cell

from aviary.api import Aircraft, Mission
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.mass.gasp_based.design_load import (
    BWBLoadSpeeds,
    BWBLoadFactors,
    LoadSpeeds,
    LoadFactors,
    BWBDesignLoadGroup,
)
from aviary.subsystems.geometry.gasp_based.wing import ExposedWing

glue_variable(get_variable_name(Aircraft.Design.WING_LOADING), md_code=True)
glue_variable(get_variable_name(Mission.Design.GROSS_MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.EXPOSED_AREA), md_code=True)

glue_variable(get_variable_name(BWBLoadSpeeds), md_code=True)
glue_variable(get_variable_name(BWBLoadFactors), md_code=True)
glue_variable(get_variable_name(LoadSpeeds), md_code=True)
glue_variable(get_variable_name(LoadFactors), md_code=True)
glue_variable(get_variable_name(BWBDesignLoadGroup), md_code=True)
glue_variable(get_variable_name(ExposedWing), md_code=True)

## BWB mass subsystem

- For design load of a BWB airctaft, variable {glue:md}`Aircraft.Design.WING_LOADING` is replaced by {glue:md}`Mission.Design.GROSS_MASS` over {glue:md}`Aircraft.Wing.EXPOSED_AREA`. As a result, {glue:md}`BWBLoadSpeeds` and {glue:md}`BWBLoadFactors` components replace {glue:md}`LoadSpeeds` and {glue:md}`LoadFactors`. A new group {glue:md}`BWBDesignLoadGroup` is created to include these two new components.
- Aviary engine geometry uses different empirical equation. In GASP, the sizing relation is based on aircraft gross weight and the number of engines. For BWB, we adopt GASP implementation. We also allow the engines that are partially buried into the fuselage. This implementation can be easily modified to apply to conventional aircraft.

In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.mass.gasp_based.equipment_and_useful_load import (
    EquipAndUsefulLoadMassGroup,
    EquipMassPartialSum,
    UsefulLoadMass,
    BWBACMass,
    BWBFurnishingMass,
)

glue_variable(get_variable_name(EquipAndUsefulLoadMassGroup), md_code=True)
glue_variable(get_variable_name(EquipMassPartialSum), md_code=True)
glue_variable(get_variable_name(UsefulLoadMass), md_code=True)
glue_variable(get_variable_name(BWBACMass), md_code=True)
glue_variable(get_variable_name(BWBFurnishingMass), md_code=True)

glue_variable(get_variable_name(Aircraft.Electrical.SYSTEM_MASS_PER_PASSENGER), md_code=True)

glue_variable(get_variable_name(Aircraft.APU.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Avionics.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.AntiIcing.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Furnishings.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.EMERGENCY_EQUIPMENT_MASS), md_code=True)

### Equip And Useful Load

  - {glue:md}`EquipAndUsefulLoadMassGroup` includes the computations of 19 items. Ideally, each of them should be done in its own component and one group has them all. This is a long time goal. For now, it is separated to two components {glue:md}`EquipMassPartialSum` and {glue:md}`UsefulLoadMass`. The air conditioning and furnishing masses are already singled out because they need to be modified for BWB.
  - A new variable {glue:md}`Aircraft.Electrical.SYSTEM_MASS_PER_PASSENGER` is added which corresponds to `CW(15)` in GASP. Its value is different for conventional aircraft and BWB.
  - Two new components {glue:md}`BWBACMass` and {glue:md}`BWBFurnishingMass` are added to `equipment_and_useful_load.py`.
  - **Note:** GASP Fortran code has new updates that are not included in Aviary. We've updated Aviary for furnishing mass but other masses need to be checked.
  - **Note:** {glue:md}`EquipMassPartialSum` has implementation errors for the computations of {glue:md}`Aircraft.APU.MASS`, {glue:md}`Aircraft.Avionics.MASS`, {glue:md}`Aircraft.AntiIcing.MASS`, {glue:md}`Aircraft.Furnishings.MASS`, and {glue:md}`Aircraft.Design.EMERGENCY_EQUIPMENT_MASS`. As a result, Aviary always uses user provided masses (not empirical formulas). We should use Aviary's feature of `overriding` to override those variables.

In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.mass.gasp_based.wing import (
    WingMassSolve,
    BWBWingMassSolve,
    BWBWingMassGroup,
    WingMassTotal,
)
from aviary.subsystems.geometry.gasp_based.wing import (
    WingFoldArea,
    WingFoldVolume,
    BWBWingFoldVolume,
    BWBWingGroup,
    BWBWingVolume,
    WingParameters,
    WingVolume,
)

glue_variable(get_variable_name(WingMassSolve), md_code=True)
glue_variable(get_variable_name(BWBWingMassSolve), md_code=True)
glue_variable(get_variable_name(BWBWingMassGroup), md_code=True)
glue_variable(get_variable_name(WingMassTotal), md_code=True)
glue_variable(get_variable_name(BWBWingVolume), md_code=True)
glue_variable(get_variable_name(WingFoldArea), md_code=True)
glue_variable(get_variable_name(WingFoldVolume), md_code=True)
glue_variable(get_variable_name(BWBWingFoldVolume), md_code=True)
glue_variable(get_variable_name(BWBWingGroup), md_code=True)
glue_variable(get_variable_name(WingParameters), md_code=True)
glue_variable(get_variable_name(WingVolume), md_code=True)

glue_variable(get_variable_name(Aircraft.Wing.SPAN), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuselage.AVG_DIAMETER), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuel.WING_VOLUME_GEOMETRIC_MAX), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.FOLDING_AREA), md_code=True)

### Wing Mass Model

  - For wing mass of BWB aircraft, variable {glue:md}`Aircraft.Wing.SPAN` has to deduct cabin width (i.e. {glue:md}`Aircraft.Fuselage.AVG_DIAMETER`). As a result, {glue:md}`WingMassSolve` component is replaced by {glue:md}`BWBWingMassSolve` component. {glue:md}`BWBWingMassGroup` is created to pair {glue:md}`BWBWingMassSolve` and {glue:md}`WingMassTotal`.
  - In `geometry/gasp_based/wing.py`, {glue:md}`Aircraft.Fuel.WING_VOLUME_GEOMETRIC_MAX` is moved out of {glue:md}`WingParameters` class. In stead, a class {glue:md}`WingVolume` is created to compute {glue:md}`Aircraft.Fuel.WING_VOLUME_GEOMETRIC_MAX`. For BWB, another component {glue:md}`BWBWingVolume` is created for the same purpose. The algorithm is quite different for BWB.
  - In `geometry/gasp_based/wing.py`, there are two components related to wing fold: {glue:md}`WingFoldArea` and {glue:md}`WingFoldVolume`. The first computes {glue:md}`Aircraft.Wing.FOLDING_AREA` and the second computes {glue:md}`Aircraft.Fuel.WING_VOLUME_GEOMETRIC_MAX`. For BWB, another class {glue:md}`BWBWingFoldVolume` is created to do the same job. Note that for BWB, {glue:md}`BWBWingFoldVolume` uses the result in {glue:md}`BWBWingVolume`.
  - A {glue:md}`BWBWingGroup` is created to put all these pieces together. 
  - Unit tests are added to make sure that the Aviary result is the same as GASP model.

In [ ]:
# Testing Cell

from aviary.api import Aircraft, Mission
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.subsystems.mass.gasp_based.fuel import (
    BWBFuselageMass,
    FuselageMass,
    StructMass,
    BodyTankCalculations,
    FuelMassGroup,
)

glue_variable(get_variable_name(BWBFuselageMass), md_code=True)
glue_variable(get_variable_name(FuselageMass), md_code=True)
glue_variable(get_variable_name(StructMass), md_code=True)
glue_variable(get_variable_name(BodyTankCalculations), md_code=True)
glue_variable(get_variable_name(FuelMassGroup), md_code=True)

glue_variable(get_variable_name(Mission.Summary.FUEL_MASS), md_code=True)
glue_variable(get_variable_name(Mission.Summary.TOTAL_FUEL_MASS), md_code=True)

glue_variable(get_variable_name(Aircraft.Design.FIXED_USEFUL_LOAD), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.HIGH_LIFT_MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuel.FUEL_SYSTEM_MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.STRUCTURE_MASS), md_code=True)
glue_variable(get_variable_name(Mission.Summary.FUEL_MASS_REQUIRED), md_code=True)
glue_variable(get_variable_name(Aircraft.Propulsion.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.Fuel.TOTAL_CAPACITY), md_code=True)
glue_variable(get_variable_name(Aircraft.Propulsion.TOTAL_ENGINE_POD_MASS), md_code=True)

### Fuel Model

  - {glue:md}`FuelMassGroup` groups all fuel related components. In case of BWB, {glue:md}`BWBFuselageMass` is in place of {glue:md}`FuselageMass`. This group has a nonlinear solver. In order for it to converge, one must provide good initial guesses for the inputs. Otherwise, it may claim that convergence is reached but gives rise to a strange solution.
  - The computation in {glue:md}`BodyTankCalculations` component can not be matched in GASP Fortran code exactly. It is possible that `extra_fuel_volume` becomes negative. We added code to make sure that it is always positive.
  - For fuselage mass of BWB aircraft, the empirical weight equation is quite different from conventional aircraft. It was computed in `FuselageAndStructMass` component. This component had two parts: fuselage mass and structural mass. In order to reuse the code for structural mass, this component is split into two components: {glue:md}`FuselageMass` and {glue:md}`StructMass`. For BWB, {glue:md}`FuselageMass` is replaced by {glue:md}`BWBFuselageMass`.
  - **Note:** The historic name of {glue:md}`Mission.Summary.FUEL_MASS_REQUIRED` is `INGASP.WFAREQ`, but `WFAREQ` includes fuel margin in GASP while {glue:md}`Mission.Summary.FUEL_MASS_REQUIRED` doesn't. The historic name of {glue:md}`Mission.Summary.TOTAL_FUEL_MASS` is `INGASP.WFA`, but does not include fuel margin in GASP while {glue:md}`Mission.Summary.FUEL_MASS_REQUIRED` does.
  - **Note:** GASP Fortran code has features that are not implemented in Aviary (e.g. tail boom support, tip tank weight, fuselage acoustic treatment, pylon, acoustic treatment).

- Comparison to GASP model

| Aviary | &nbsp; &nbsp; | GASP | &nbsp; &nbsp; | Observation |
| ------- | ------- | ------- | -------- | ------------- |
| {glue:md}`Aircraft.Design.FIXED_USEFUL_LOAD` | 5972 | WFUL | 5775 | different unit weight of pilots and attendents |
| {glue:md}`Aircraft.Wing.HIGH_LIFT_MASS` | 972 | WHLDEV | 974 | In GASP, wing loading is a variable, but in Aviary, it is a constant |
| {glue:md}`Aircraft.Fuel.FUEL_SYSTEM_MASS` | 1316 | WFSS | 1281 | the mass in GASP is computed after engine sizing. |
| {glue:md}`Aircraft.Design.STRUCTURE_MASS` | 44471 | WST | 45623 | the mass in GASP is computed after engine sizing. |
| {glue:md}`Mission.Summary.FUEL_MASS` | 34188 | WFADES | 33268 | The mass in GASP is computed after engine sizing. |
| {glue:md}`Aircraft.Propulsion.MASS` | 8628 | WP | 8592 | the mass in GASP is computed after engine sizing. |
| {glue:md}`Aircraft.Fuel.TOTAL_CAPACITY` | 38068 | WFAMAX | 33268 | The mass in GASP is computed after engine sizing. |
| {glue:md}`Aircraft.Propulsion.TOTAL_ENGINE_POD_MASS` | 1687 | WPES | the mass in GASP is computed after engine sizing. |
| {glue:md}`Mission.Summary.FUEL_MASS_REQUIRED` | 34188 | WFAREQ | 36595 | Aviary does not include margin. |
| | | | |

Because most of the variables match pretty well. We show those with significant differences. As we see, most differences are due to the fact that GASP computes the fuel masses after engine sizing.

The fuel computation is a nonlinear system of equations. A simplied XDSM diagram is shown below:

![GASP based fuel mass computation](images/gasp_bwb_fuel_mass.png)

A Newton solver is applied until {glue:md}`Mission.Summary.FUEL_MASS`, `wingfuel_mass_min` and {glue:md}`Aircraft.Fuel.TOTAL_CAPACITY` are converged.

For conventional aircraft, {glue:md}`BWBFuselageMass` is replaced by {glue:md}`FuselageMass`.

In [ ]:
# Testing Cell

from aviary.utils.doctape import get_all_non_aviary_names, get_variable_name, glue_variable
from aviary.subsystems.aerodynamics.gasp_based.gaspaero import (
    BWBBodyLiftCurveSlope,
    BWBFormFactorAndSIWB,
    BWBAeroSetup,
    BWBLiftCoeff,
    BWBLiftCoeffClean,
    AeroGeom,
    Xlifts,
)
from aviary.subsystems.aerodynamics.gasp_based.common import AeroForces
from aviary.subsystems.aerodynamics.gasp_based.gaspaero import CruiseAero, LowSpeedAero
from aviary.subsystems.aerodynamics.gasp_based.table_based import TabularCruiseAero

glue_variable(get_variable_name(BWBBodyLiftCurveSlope), md_code=True)
glue_variable(get_variable_name(BWBFormFactorAndSIWB), md_code=True)
glue_variable(get_variable_name(BWBAeroSetup), md_code=True)
glue_variable(get_variable_name(BWBLiftCoeff), md_code=True)
glue_variable(get_variable_name(BWBLiftCoeffClean), md_code=True)
glue_variable(get_variable_name(CruiseAero), md_code=True)
glue_variable(get_variable_name(LowSpeedAero), md_code=True)
glue_variable(get_variable_name(AeroGeom), md_code=True)
glue_variable(get_variable_name(TabularCruiseAero), md_code=True)

lift_curve_slope, lift_ratio = get_all_non_aviary_names(Xlifts, include_in_out='out')
glue_variable('lift_curve_slope', lift_curve_slope, md_code=False)
glue_variable('lift_ratio', lift_ratio, md_code=False)
sa1, sa2, sa3, sa4, sa5, sa6, sa7, cf = get_all_non_aviary_names(AeroGeom, include_in_out='out')
glue_variable('SA1', sa1, md_code=False)
glue_variable('SA2', sa2, md_code=False)
glue_variable('SA3', sa3, md_code=False)
glue_variable('SA4', sa4, md_code=False)
glue_variable('SA5', sa5, md_code=False)
glue_variable('SA6', sa6, md_code=False)
glue_variable('SA7', sa7, md_code=False)
glue_variable('cf', cf, md_code=False)
[cl, cd] = get_all_non_aviary_names(AeroForces, include_in_out='in')
glue_variable('CL', cl, md_code=False)
glue_variable('CD', cd, md_code=False)

## BWB aerodynamics

This feature implements GASP aerodynamics subsystems for BWB aircraft. Five new components are added:

- {glue:md}`BWBBodyLiftCurveSlope`
- {glue:md}`BWBFormFactorAndSIWB`
- {glue:md}`BWBAeroSetup`
- {glue:md}`BWBLiftCoeff`
- {glue:md}`BWBLiftCoeffClean`

Two group components {glue:md}`CruiseAero` and {glue:md}`LowSpeedAero` are configured for BWB as an option (default to conventional aircraft).

In GASP, friction due to nacelle is removed from the computation of `SA5`. Instead, it is computed separately and is added in the drag computation. The Aviary implementation is not the same.

`alpha_stall` and `CL_max` are computed based on wing only for now. We expect that they will be updated in the future.

Table based aerodynamics is still available to BWB as long as users provide aerodynamics tables. A sample cruise aero table for BWB is available and an example using {glue:md}`TabularCruiseAero` is provided.

### Comparison of `BWBAeroSetup` with GASP

| Variables | GASP | Variables | Aviary |
| ---------- | ------ | ------- | ------- |
| CLAW | 4.63868 | {glue:md}`lift_curve_slope` | 4.63868 |
| BARL | -0.14081 | {glue:md}`lift_ratio` | -0.14081 |
| CFIN | 0.002836 | {glue:md}`cf` | 0.002836 |
| SA1 | 0.81401 | {glue:md}`SA1` | 0.80832 |
| SA2 | -0.15743 | {glue:md}`SA2` | -0.13651 |
| SA3 | 0.033989 | {glue:md}`SA3` | 0.033989 |
| SA4 | 0.10197 | {glue:md}`SA4` | 0.10197 |
| SA5 | 0.004464 | {glue:md}`SA5` | 0.009628 |
| SA6 | 2.23877 | {glue:md}`SA6` | 2.09277 |
| SA7 | 0.034136 | {glue:md}`SA7` | 0.040498 |

The differences are due to several reasons:

- GASP has different coefficients of friction for different part of an aircraft. For this purpose, several new parameters (aero calibration factors) are added. Aviary has one single coefficient {glue:md}`cf` (an output from {glue:md}`AeroGeom` component)
- GASP has several factors that are included in the computation of friction (e.g. winglet, tip tank, excrescence) but not in Aviary.
- GASP excludes frictions from nacelle in {glue:md}`SA5`. Nacelle friction is done in engine computation and is added in drag computation. But in Aviary, nacelle friction is included in {glue:md}`SA5` and not in drag computation.

### Comparison of `CruiseAero` with GASP

| Variables | GASP | Variables | Aviary |
| ---------- | ------ | ------- | ------- |
| CLTOT | 0.41069 | {glue:md}`CL` | 0.41067 |
| CD | 0.014738 | {glue:md}`CD` | 0.022509 |
| CL/CD | 27.86518 | L/D | 18.24451 |

As we see, `CL` matches closely but `CD` doesn't. This is because the differences in {glue:md}`BWBAeroSetup` as we discussed above.

### Comparison of `LowSpeedAero` with GASP

| Variables (Takeoff) | CL (GASP/Aviary) | CD (GASP/Aviary)| CL/CD (GASP/Aviary) |
| --------------- | -----------------  | ------------------- | ----------------------- |
| α = -2.0 | 0.07507 / 0.05787 | 0.01853 / 0.02565 | 4.05136 / 3.307513 |
| α = 0.0 | 0.23964 / 0.21906 | 0.01866 / 0.02592 | 12.84433 / 9,49165 |
| α = 2.0 | 0.40422 / 0.407231 | 0.02070 / 0.02844 | 20.74018 / 16.22583 |

| Variables (Landing) | CL (GASP/Aviary) | CD (GASP/Aviary)| CL/CD (GASP/Aviary) |
| --------------- | -----------------  | ------------------- | ----------------------- |
| α = -2.0 | 0.18551 / 0.19824 | 0.02299 / 0.02962 | 8.06918 / 6.69194 |
| α = 0.0 | 0.35009 / 0.35944 | 0.02292 / 0.02970 | 15.27225 / 12.10145 |
| α = 2.0 | 0.51467 / 0.52062 | 0.02482 / 0.03209 | 20.74018 / 16.22583 |

As we see, `CL` matches closely but `CD` doesn't. This is because the differences in  {glue:md}`BWBAeroSetup`.

## Missing features in Aviary

In addition to the missing fetures, there are other features in GASP that are not implemented in Aviary.

- GASP computes maximum CL for cruise, take-off, and landing phases but not in Aviary.
- GASP computes lift curve slope (i.e. the derivative of the Lift Coeff w.r.t. Alpha), named `CLATOT`. 
- GASP computes stall alpha from the wings. For BWB, this is not sufficient. Both GASP and Aviary should enhance their models.
- GASP computes excrescence drag.
- Drag coefficients SA3 and SA4 are computed in Aviary but are not used.
- Aviary does not have tail boom support.
- Aviary does not have winglet geometry.
- In GASP, a pilot weight is 170 lb and in Aviary it is 198 lb. In GASP, each attendant weights 130 lb and in Aviary it is 177 lb.
- GASP has fuselage acoustic treatment.
- GASP conputes tip tank weight.
- GASP allows canard configurations.
